<a href="https://colab.research.google.com/github/allisonlinn/CSUREMM/blob/main/shorter_time_frame_analysis_w_dataframe_(granger%2C_pearson%2C_overlap).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

# Upload CSV file from local machine
uploaded = files.upload()

# Get the file name
file_name = next(iter(uploaded))

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_name)

Saving final odessy_ our detrended data - detrended_final (11).csv to final odessy_ our detrended data - detrended_final (11).csv


In [2]:
from scipy.stats import pearsonr
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd
from google.colab import files

# Create an empty DataFrame to store the statistically significant results
significant_results = pd.DataFrame(columns=['year', 'season', 'fund name', 'p-value', 'correlation coefficient'])

# Create an empty DataFrame to store the Granger causality test results
granger_results = pd.DataFrame(columns=['year', 'season', 'fund name', 'lag', 'p-value'])

# Get unique combinations of year and season
year_season_combos = df[['year', 'season']].drop_duplicates()

# Iterate over each year-season combo
for index, combo in year_season_combos.iterrows():
    year = combo['year']
    season = combo['season']

    # Filter data for the current year-season combo
    filtered_data = df[(df['year'] == year) & (df['season'] == season)]

    # Define the list of columns to calculate correlations and perform Granger causality test with
    columns_to_analyze = ['VCDAX_V_N', 'VCSAX_V_N', 'VENAX_V_N', 'VFAIX_V_N', 'VGSLX_V_N', 'VCHCX_V_N', 'VITAX_V_N', 'VMAIX_V_N', 'VSPVX_V_N', 'VTCAX_V_N', 'VUIAX_V_N', 'VINAX_V_N', 'VUIAX_P_N', 'VGHCX_P_N', 'VGHCX_P_N', 'VSPVX_P_N', 'VITAX_P_N', 'VMIAX_P_N', 'VGSLX_P_N', 'VINAX_P_N', 'VTCAX_P_N', 'VCSAX_P_N', 'VENAX_P_N', 'VCDAX_P_N']

    # Perform Pearson correlation analysis and Granger causality test for each column
    for column in columns_to_analyze:
        try:
            # Extract the relevant series and the target variable
            X = filtered_data['compound']
            y = filtered_data[column]

            # Calculate Pearson correlation coefficient and p-value
            correlation, p_value = pearsonr(X, y)

            # Check if the correlation is statistically significant
            if p_value < 0.05:
                # Add the significant correlation result to the DataFrame
                result = {
                    'year': year,
                    'season': season,
                    'fund name': column,
                    'p-value': p_value,
                    'correlation coefficient': correlation
                }
                significant_results = pd.concat([significant_results, pd.DataFrame(result, index=[0])])

            # Perform Granger causality test with different lags
            max_lag = 5  # Maximum lag to test
            granger_test_results = grangercausalitytests(filtered_data[['compound', column]], maxlag=max_lag, verbose=False)

            # Iterate over the lag values and add significant Granger causality results to the DataFrame
            for lag in range(1, max_lag + 1):
                p_value = granger_test_results[lag][0]['ssr_ftest'][1]
                if p_value < 0.05:
                    result = {
                        'year': year,
                        'season': season,
                        'fund name': column,
                        'lag': lag,
                        'p-value': p_value
                    }
                    granger_results = pd.concat([granger_results, pd.DataFrame(result, index=[0])])
        except ValueError:
            # Handle the ValueError when x and y have length less than 2
            pass

# Export the significant results DataFrame to a CSV file
significant_results.to_csv('significant_results.csv', index=False)
files.download('significant_results.csv')

# Export the Granger causality test results DataFrame to a CSV file
granger_results.to_csv('granger_results.csv', index=False)
files.download('granger_results.csv')

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 1
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 4
  warnings.warn('covariance of constraints does not have full '
/usr/local/l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
# Cross-reference the season-year-fund combos from both tests
cross_referenced_combos = pd.merge(significant_results, granger_results, on=['year', 'season', 'fund name'], how='inner')

# Export the cross-referenced combos DataFrame to a CSV file
cross_referenced_combos.to_csv('cross_referenced_combos.csv', index=False)
files.download('cross_referenced_combos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>